In [25]:
import pandas as pd

In [26]:
df_mid = pd.read_csv("pythia_93k.csv")
df_full = pd.read_csv("pythia_full.csv")

In [27]:
df_mid["70M"]

0    0.214 + 0.006
1    0.598 + 0.011
2    0.508 + 0.014
3    0.365 + 0.047
4    0.359 + 0.010
5    0.172 + 0.011
6    0.642 + 0.015
7    0.220 + 0.016
Name: 70M, dtype: object

In [28]:
def process_dataframe(df):
    for col in df.columns:
        if col != 'Task':
            df[col] = df[col].str.split().str[0].astype(float)
    return df

In [29]:
df_mid = process_dataframe(df_mid)
df_full = process_dataframe(df_full)

In [30]:
df_full.set_index('Task', inplace=True)
df_mid.set_index('Task', inplace=True)

In [34]:
df_diff = df_full.subtract(df_mid)

In [35]:
df_diff

,70M,160M,410M,1B,1.4B,2.8B,6.9B,12B
Task,,,,,,,,
Lambada (OpenAI),-0.029,-0.040,0.016,0.013,0.024,0.014,0.016,0.021
PIQA,-0.003,0.002,0.001,0.006,0.006,0.008,0.011,0.005
WinoGrande,0.020,0.019,0.012,0.018,0.013,0.002,0.016,0.009
WSC,0.000,0.000,-0.058,0.000,-0.029,0.020,0.000,-0.087
ARC - Easy,0.015,-0.028,0.009,0.018,0.012,0.022,0.016,0.016
ARC - Challenge,0.009,-0.004,-0.005,0.015,0.007,0.014,-0.005,0.006
SciQ,-0.041,-0.023,0.003,0.003,-0.008,0.007,-0.004,-0.007
LogiQA,-0.010,-0.024,0.014,-0.001,-0.014,-0.008,0.013,-0.006


In [36]:
import numpy as np

slopes = {}
for task in df_diff.index:
    x = np.arange(len(df_diff.columns))
    y = df_diff.loc[task].values
    slope, _ = np.polyfit(x, y, deg=1)
    slopes[task] = slope

# Create a DataFrame with slopes
slope_df = pd.DataFrame(list(slopes.items()), columns=['Task', 'Slope'])

In [40]:
slope_df.sort_values(by='Slope',ascending=False)

,Task,Slope
0,Lambada (OpenAI),0.007560
6,SciQ,0.003976
4,ARC - Easy,0.003095
7,LogiQA,0.001595
1,PIQA,0.001452
5,ARC - Challenge,0.000274
2,WinoGrande,-0.001512
3,WSC,-0.004810
